# Columnar analysis and Awkward Array

<br><br><br>

## Introduction

"Columnar" is an overloaded term. I've used it in two different ways:

* arranging data in memory or on disk in ways that allow faster and selective readout (some types of TTree data have been columnar since 1995)
* performing computations explicitly on arrays of data, rather than scalar values: "no for loops!"

Only the second one directly impacts physicists doing analysis.

I prefer calling it "array-oriented programming," as a programming paradigm like "imperative" and "functional."

<br><br><br>

### Easy example of imperative, functional, and array-oriented

Compute the square of every element in a list/array.

<br><br><br>

#### Imperative

In [ ]:
original = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

result = []
for x in original:
    result.append(x**2)

result

<br><br><br>

#### Functional

In [ ]:
original = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

result = list(map(lambda x: x**2, original))

result

Functional programming with `map` isn't common in Python, but list comprehensions are pretty close to the "spirit" of functional programming:

In [ ]:
original = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

result = [x**2 for x in original]

result

<br><br><br>

#### Array-oriented

In [ ]:
import numpy as np

In [ ]:
original = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

result = original**2

result

<br><br><br>

### Hard example of imperative, functional, and array-oriented

Compute gravitational forces among $n$ particles in 3 dimensions.

In [ ]:
m = np.array([100, 1, 1])   # sun and a double-planet (a 3-body problem)

# initial position (x) and momentum (p)
x = np.array([[0, 0, 0], [0, 0.9, 0], [0, 1.1, 0]])
p = np.array([[0, 0, 0], [-13, 0, 0], [-10, 0, 0]])

G = 1

<br><br><br>

#### Imperative

In [ ]:
def imperative_forces(m, x, p):
    total_force = np.zeros_like(x)

    for i in range(len(x)):
        for j in range(i + 1, len(x)):
            if i != j:
                mi, mj = m[i], m[j]
                xi, xj = x[i], x[j]
                pi, pj = p[i], p[j]
                displacement = [
                    xj[0] - xi[0],
                    xj[1] - xi[1],
                    xj[2] - xi[2],
                ]
                distance = np.sqrt(displacement[0]**2 + displacement[1]**2 + displacement[2]**2)
                direction = [
                    displacement[0] / distance,
                    displacement[1] / distance,
                    displacement[2] / distance,
                ]
                force = [
                    G * mi * mj * direction[0] / distance**2,
                    G * mi * mj * direction[1] / distance**2,
                    G * mi * mj * direction[2] / distance**2,
                ]
                total_force[i, 0] += force[0]
                total_force[i, 1] += force[1]
                total_force[i, 2] += force[2]
                total_force[j, 0] += -force[0]
                total_force[j, 1] += -force[1]
                total_force[j, 2] += -force[2]

    return total_force

<br><br><br>

#### Functional

In [ ]:
from functools import reduce
from itertools import combinations

In [ ]:
def functional_forces(m, x, p):
    def negate(vector):
        return [-a for a in vector]

    def add(*vectors):
        return [reduce(lambda a, b: a + b, components) for components in zip(*vectors)]

    def subtract(vectorA, vectorB):
        return add(vectorA, negate(vectorB))

    def magnitude(vector):
        return np.sqrt(reduce(lambda a, b: a + b, map(lambda a: a**2, vector)))

    def force(mi, mj, xi, xj, pi, pj):
        displacement = subtract(xi, xj)
        distance = magnitude(displacement)
        direction = [a / distance for a in displacement]
        return [G * mi * mj * a / distance**2 for a in direction]

    pairwise_forces = [
        ((i, j), force(mi, mj, xi, xj, pi, pj))
        for ((i, (mi, xi, pi)), (j, (mj, xj, pj))) in combinations(enumerate(zip(m, x, p)), 2)
    ]

    def partial_forces(pairwise_forces, i):
        return (
            [force for (_, check), force in pairwise_forces if i == check] +
            [negate(force) for (check, _), force in pairwise_forces if i == check]
        )

    return np.array([add(*partial_forces(pairwise_forces, i)) for i in range(len(m))])

<br><br><br>

#### Array-oriented

In [ ]:
def array_forces(m, x, p):
    i, j = np.triu_indices(len(x), k=1)
    pw_displacement = x[j] - x[i]
    pw_distance = np.sqrt(np.sum(pw_displacement**2, axis=-1))
    pw_direction = pw_displacement / pw_distance[:, np.newaxis]
    pw_force = G * m[i, np.newaxis] * m[j, np.newaxis] * pw_direction / pw_distance[:, np.newaxis]**2
    total_force = np.zeros_like(x)
    np.add.at(total_force, i, pw_force)
    np.add.at(total_force, j, -pw_force)
    return total_force

<br><br><br>

In [ ]:
imperative_forces(m, x, p)

In [ ]:
functional_forces(m, x, p)

In [ ]:
array_forces(m, x, p)

<br><br><br>

#### Let's see it!

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
def array_step(m, x, p, dt):
    # this is a numerically stable way of updating positions, momenta, and forces
    p += array_forces(m, x, p) * (dt/2)    # half kick
    x += p * dt / m[:, np.newaxis]         # full drift
    p += array_forces(m, x, p) * (dt/2)    # half kick

In [ ]:
def plot(m, x, p, dt, num_frames=100, steps_per_frame=10):
    num_particles = len(m)

    history = np.empty((num_frames, num_particles, 2))
    for i in range(num_frames):
        history[i, :, 0] = x[:, 0]
        history[i, :, 1] = x[:, 1]
        for _ in range(steps_per_frame):
            array_step(m, x, p, dt)

    fig, ax = plt.subplots(figsize=(5, 5))

    lines = []
    for j in range(num_particles):
        lines.append(ax.plot(history[:1, j, 0], history[:1, j, 1])[0])
    dots = ax.scatter(history[0, :, 0], history[0, :, 1])

    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)

    def update(i):
        for j, line in enumerate(lines):
            line.set_xdata(history[:i, j, 0])
            line.set_ydata(history[:i, j, 1])
        dots.set_offsets(history[i, :, :])
        return [*lines, dots]

    ani = animation.FuncAnimation(fig=fig, func=update, frames=num_frames, interval=50, blit=True)
    out = HTML(ani.to_jshtml())
    plt.close()
    return out

In [ ]:
m = np.array([100, 1, 1], np.float64)
x = np.array([[0, 0, 0], [0, 0.9, 0], [0, 1.1, 0]], np.float64)
p = np.array([[0, 0, 0], [-13, 0, 0], [-10, 0, 0]], np.float64)

plot(m, x, p, dt=0.001)

In [ ]:
a = 0.347111
b = 0.532728
m = np.array([1, 1, 1], np.float64)
x = np.array([[-1, 0, 0], [1, 0, 0], [0, 0, 0]], np.float64)
p = np.array([[a, b, 0], [a, b, 0], [-2 * a, -2 * b, 0]], np.float64)

plot(m, x, p, dt=0.01)

In [ ]:
m = np.ones(25)
x = np.random.normal(0, 1, (25, 3))
p = np.random.normal(0, 1, (25, 3))

plot(m, x, p, dt=0.0025)

<br><br><br>

#### Let's time it!

In [ ]:
m = np.ones(500)
x = np.random.normal(0, 1, (500, 3))
p = np.random.normal(0, 1, (500, 3))

In [ ]:
%%timeit -n1 -r1

imperative_forces(m, x, p)

In [ ]:
%%timeit -n1 -r1

functional_forces(m, x, p)

In [ ]:
%%timeit -n1 -r1

array_forces(m, x, p)

<br><br><br>

In Python, array-oriented programming is a big advantage because it avoids Python's overhead (virtual machine, dynamic data types, garbage collection, etc).

* In `imperative_forces`, each of the 500 × 499 pairs of particles has to step through 25 lines of Python code (214 byte-code instructions).
* In `functional_forces`, each of the 500 × 499 pairs of particles has to step through functions that add up to 12 lines of Python code (100 byte-code instructions), plus call-stack overhead.
* `array_forces` has 9 lines of Python code (103 byte-code instructions), but the Python virtual machine steps through them only once, _not once per pair of particles_.

<br><br><br>

This is also relevant for GPU programming.

To get the most performance out of GPU programming frameworks like CUDA, you need to arrange the computation as array-at-a-time and think about vectorization.

<br><br><br>

### What if imperative code is easier to reason about?

Sometimes it is.

If it's easier to think about a problem imperatively, but the loop would iterate over some large number, just make sure Python isn't implementing the loop.

* Just-In-Time (JIT) compile it with [Numba](https://numba.pydata.org/).
* Use ROOT's [RDataFrame](https://root.cern/doc/master/classROOT_1_1RDataFrame.html) to compile and run C++ over ROOT data.
* Use [cppyy](https://cppyy.readthedocs.io/) to compile and run C++ over arbitrary data.
* Use [Julia](https://julialang.org/).
* Use [pybind11](https://pybind11.readthedocs.io/) to compile a Python extension in C++ or [PyO3](https://pyo3.rs/) to compile a Python extension in Rust.

All of these involve more set-up time to get started than array-oriented programming, but may be easier to deal with in the long run, depending on the problem.

<br><br><br>

## This tutorial

This tutorial will be about array-oriented programming in Python.

Exploring and analyzing data in an array-oriented way is a useful skill.

It's organized as a set of puzzles that we'll solve together. Open [student.ipynb](student.ipynb) and test the `send_answer` function now.

In [ ]:
from communication import collect_answers

In [ ]:
collect_answers()

<br><br><br>

## Puzzles

### NumPy puzzle 1

Given a 3D array,

In [ ]:
import numpy as np

In [ ]:
array3d = np.arange(2 * 3 * 5).reshape(2, 3, 5)
array3d

you can select items

<img src="img/array3d-highlight1.svg" width="50%">

with

In [ ]:
array3d[:, 1:, 1:]

Write a slice the selects these elements:

<img src="img/array3d-highlight2.svg" width="50%">

In [ ]:
collect_answers()

<br><br><br>

### NumPy puzzle 2

Compute the size of the spaces between consecutive elements in the following array.

In [ ]:
array = np.array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
array

**Hint:**

<img src="img/flat-operation.svg" width="50%"><img src="img/shifted-operation.svg" width="50%">

In [ ]:
collect_answers()

<br><br><br>

### NumPy puzzle 3

Compute the length of this curve.

<img src="img/length-by-segment.svg" width="50%">

In [ ]:
t = np.linspace(0, 2*np.pi, 10000)
x = np.sin(3*t)
y = np.sin(4*t)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x, y);

In [ ]:
collect_answers()

<br><br><br>

### NumPy puzzle 4

Scale this image down by a factor of 64 on both sides, using only [np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html), [np.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html), and [np.ndarray.astype](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html).

In [ ]:
import matplotlib.image

In [ ]:
image = matplotlib.image.imread("data/sun-shines-in-CMS.jpg")
plt.imshow(image);

The current shape is

In [ ]:
image.shape

1920 rows, 2560 columns, and the third axis is for (red, green, blue), all `np.uint8`.

Your strategy should be to reshape the array, such that the dimension of length `1920` becomes two new dimensions of length `1920 // 64` and `64` and the dimension of length `2560` becomes two new dimensions of length `2560 // 64` and `64`. Then average over each of the dimensions of length `64`.

The shape should change as

$$\left(1920, 2560, 3\right) \to \left(\frac{1920}{64}, 64, \frac{2560}{64}, 64, 3\right) \to \left(\frac{1920}{64}, \frac{2560}{64}, 3\right)$$

and then you need to turn the floating-point dtype back into unsigned 8-bit integers with [np.ndarray.astype](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html).

In [ ]:
collect_answers()

<br><br><br>

### NumPy puzzle 5

Let's interpret some raw ROOT data!

In [ ]:
import zlib

In [ ]:
with open("data/SMHiggsToZZTo4L.root", "rb") as file:
    file.seek(42104123)
    compressed_data = file.read(14718)
    uncompressed_data = uncompressed_data = zlib.decompress(compressed_data)
    array_of_uint8 = np.frombuffer(uncompressed_data, np.uint8, 12524)

In [ ]:
array_of_uint8

This should be 3131 floating-point muon $p_T$ values.

But if we view the data as `float32`, the orders of magnitude are all wrong:

In [ ]:
array_of_uint8.view(np.float32)

That's because the data are [big endian](https://en.wikipedia.org/wiki/Endianness) and this computer is little-endian.

NumPy has a way to view "wrong"-endian data, but operations on it aren't as efficient.

In [ ]:
array_of_uint8.view(">f4")

To fix the endianness, you'll need to reverse the order of bytes _in groups of 4_.

<img src="img/big-little-endian.svg" width="75%">

Fix the endianness with only [np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) and a slice.

In [ ]:
collect_answers()

<br><br><br>

### Introduction to ragged arrays

In [ ]:
import awkward as ak

In [ ]:
array2d = np.array([
    [  1,   2,   3,   4],
    [ 10,  20,  30,  40],
    [100, 200, 300, 400],
])
array2d

In [ ]:
ragged = ak.Array([
    [   1,    2,    3,   4],
    [  10, None,   30     ],
    [ 100,  200           ],
])
ragged

Some operations, like summation along an axis, need to be generalized.

<img src="img/example-reducer-2d.svg" width="50%"><img src="img/example-reducer-ragged.svg" width="50%">

In [ ]:
np.sum(array2d, axis=0)

In [ ]:
np.sum(array2d, axis=1)

In [ ]:
np.sum(ragged, axis=0)

In [ ]:
np.sum(ragged, axis=1)

Particle physics data naturally comes in ragged arrays.

In [ ]:
import uproot

In [ ]:
with uproot.open("data/SMHiggsToZZTo4L.root") as file:
    dataset = file["Events"].arrays()

In [ ]:
dataset

In [ ]:
dataset.Electron_pt

Almost always, `axis=0` is the _events_ axis (collision events), and `axis=1` is the _particle_ axis (electrons, muons, jets...).

Let's make this easier to use by defining:

In [ ]:
EVENT = 0
PARTICLE = 1

In [ ]:
ak.num(dataset, axis=EVENT)

The number of events is 299973. (See documentation for [ak.num](https://awkward-array.org/doc/main/reference/generated/ak.num.html).)

In [ ]:
ak.num(dataset.Electron_pt, axis=PARTICLE)

The number of electrons is 0 (in the first event), 4 (in the second event), 2 (in the third event), ... 2 (in the last event).

The following computes the average electron $p_T$ in each event:

In [ ]:
np.mean(dataset.Electron_pt, axis=PARTICLE)

**Question:** what is the following? (What does the calculation mean?)

In [ ]:
np.mean(dataset.Electron_pt, axis=EVENT)

<br><br><br>

### Awkward puzzle 1

Are the muon $p_T$ values sorted within each event? That is, is it true that

```python
dataset.Muon_pt[i, j] > dataset.Muon_pt[i, j + 1]
```

for all `i` and `j`? Perform a calculation that proves it.

<details>
    <summary style="font-weight: bold;">Hint!</summary>

<p>Think about NumPy puzzle 2.</p>

</details>

In [ ]:
collect_answers()

<br><br><br>

### Awkward puzzle 2

Using [ak.num](https://awkward-array.org/doc/main/reference/generated/ak.num.html), select events with exactly two muons and make separate arrays of each of those two muons.

<details>
    <summary style="font-weight: bold;">Hint!</summary>

You can select elements from an array by slicing with a boolean array. `True` keeps an item, `False` discards it.

```python
dataset[array_of_booleans]
```

</details>

In [ ]:
collect_answers()

<br><br><br>

### Awkward puzzle 3

The `dataset` has separate ragged arrays for each particle attribute, such as `Electron_pt`, `Electron_phi`, `Electron_eta`, `Electron_mass`. It would be nicer to work with as arrays of electron _objects_ with Lorentz vector methods.

We can do that with [ak.zip](https://awkward-array.org/doc/main/reference/generated/ak.zip.html) and the [Vector](https://vector.readthedocs.io/) library.

In [ ]:
import vector
vector.register_awkward()

In [ ]:
electrons = ak.zip({
    "pt": dataset.Electron_pt,
    "phi": dataset.Electron_phi,
    "eta": dataset.Electron_eta,
    "mass": dataset.Electron_mass,
    "charge": dataset.Electron_charge,
}, with_name="Momentum4D")

In [ ]:
muons = ak.zip({
    "pt": dataset.Muon_pt,
    "phi": dataset.Muon_phi,
    "eta": dataset.Muon_eta,
    "mass": dataset.Muon_mass,
    "charge": dataset.Muon_charge,
}, with_name="Momentum4D")

In [ ]:
electrons.rapidity

You're ready to compute dimuon masses.

1. Select events with exactly two muons.
2. In each event, add those masses together with `+`.
3. Get the `mass` property from the Lorentz-added vectors.

It should look like

```
[87.1,
 90.5,
 89.2,
 18.8,
 4.59,
 37.2,
 90.6,
 8.91,
 18.1,
 27.5,
 ...,
 31.2,
 0.762,
 14.3,
 90.7,
 90,
 88.6,
 88.8,
 27.9,
 90.5]
---------------------
type: 85838 * float32
```

In [ ]:
collect_answers()

In [ ]:
Hist.new.Reg(100, 0, 100, name="mass (GeV)").Double().fill(
    ...
).plot();

<br><br><br>

### Awkward puzzle 4